In [1]:
import boto3

In [2]:
s3 = boto3.resource('s3',
 aws_access_key_id='',
 aws_secret_access_key=''
)

In [3]:
try:
 s3.create_bucket(Bucket='datacont-piper', CreateBucketConfiguration={'LocationConstraint': 'us-west-2'})
except:
 print("this may already exist")

this may already exist


In [4]:
bucket = s3.Bucket("datacont-piper")

In [5]:
bucket.Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'AFEBE5052F144763',
  'HostId': 'ZNCsv4PbJjeshRrncagj+wWmLdYn+1PvtDS3VlQ/SHOvauEtfLMdFY5S3lun9mXRdqg+lU15v2U=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ZNCsv4PbJjeshRrncagj+wWmLdYn+1PvtDS3VlQ/SHOvauEtfLMdFY5S3lun9mXRdqg+lU15v2U=',
   'x-amz-request-id': 'AFEBE5052F144763',
   'date': 'Mon, 05 Oct 2020 01:35:37 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [6]:
#upload a new object into the bucket
body = open('/Users/sue/Public/cs1660_CC/test-s3-upload.java', 'rb')

In [7]:
o = s3.Object('datacont-piper', 'test').put(Body=body )

In [8]:
body = open('/Users/sue/Public/cs1660_CC/experiments.csv', 'rb')
o = s3.Object('datacont-piper', 'testcsv').put(Body=body )

In [9]:
s3.Object('datacont-piper', 'test').Acl().put(ACL='public-read')
s3.Object('datacont-piper', 'testcsv').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': '232616D37BC375FF',
  'HostId': '8EolREd+oWMA7M7Hh1wC1JeUVbyPzhXs9btYwnqAyszFxS7m+09TMrUdery9EretH0WpmU9vu7s=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '8EolREd+oWMA7M7Hh1wC1JeUVbyPzhXs9btYwnqAyszFxS7m+09TMrUdery9EretH0WpmU9vu7s=',
   'x-amz-request-id': '232616D37BC375FF',
   'date': 'Mon, 05 Oct 2020 01:35:39 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [10]:
dyndb = boto3.resource('dynamodb',
 region_name='us-west-2',
 aws_access_key_id='',
 aws_secret_access_key=''
)

In [11]:
try:
    table = dyndb.create_table(
        TableName='myTable1',
        KeySchema=[
            {
                'AttributeName': 'PartitionKey',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'RowKey',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'PartitionKey',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'RowKey',
                'AttributeType': 'S'
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
except:
    #if there is an exception, the table may already exist. if so...
    table = dyndb.Table("myTable1")
    
table.meta.client.get_waiter('table_exists').wait(TableName='myTable1')  

In [12]:
print(table.item_count)

0


In [13]:
import csv

In [14]:
with open('/Users/sue/Public/cs1660_CC/experiments.csv', 'r', newline='') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    for item in csvf:
        print(item)

        metadata_item = {
            'PartitionKey': item[0], 
            'RowKey': item[1], 
            'description' : item[4], 
            'date' : item[2]
        }

        try:
            table.put_item(Item=metadata_item)
        except:
            print("item may already be there or another failure")

['\ufeffexperiment1', '1', '3/15/2002', 'exp1', 'this is the comment']
['experiment2', '2', '3/15/2002', 'exp2', 'this is the comment2']
['experiment3', '3', '3/16/2002', 'exp3', 'this is the comment3']
['experiment4', '4', '3/16/2002', 'exp4', 'this is the comment233']


In [15]:
response = table.get_item(
 Key={
 'PartitionKey': 'experiment1',
 'RowKey': '4'
 }
)

In [16]:
response

{'ResponseMetadata': {'RequestId': 'RK5K515D4RAOQKB94MF7KLUA5NVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 05 Oct 2020 01:35:39 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'RK5K515D4RAOQKB94MF7KLUA5NVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}